In [28]:
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation

from explaneat.core.neuralneat import NeuralNeat as NeuralNeat

import random

import torch
import torch.nn as nn
import torch.optim as optim

import neat

from explaneat.core.utility import MethodTimer


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
# from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter

from explaneat.core.neuralneat import NeuralNeat as nneat

import torch.optim as optim

from torch.functional import F








In [29]:
RANDOM_SEED      = 42
torch.manual_seed(RANDOM_SEED)


USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")


In [30]:

def xor(a, b, threshold = 0.5):
    response = False
    if a > threshold and b < threshold:
        response = True
    if a < threshold and b > threshold:
        response = True
    # return (1.0, 0.0) if response else (0.0, 1.0)
    return 1.0 if response else 0.0
    

def create_n_points(n, size, min=0.0, max=1.0):
    data = []
    for _ in range(n):
        data.append([
            random.uniform(min, max) for ii in range(size)
        ])

    return data

# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

def overUnder(val, threshold):
    return 1. if val > threshold else 0

xor_inputs = create_n_points(400, 2, -1, 1)

xor_outputs = [
    [xor(tup[0], tup[1], 0)] for tup in xor_inputs
]

In [27]:
config_path = "./config_xor"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    with MethodTimer("Backprop everything"):
        p = BackpropPopulation(config, 
                                xs, 
                                ys, 
                                criterion=nn.BCEWithLogitsLoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

def eval_genomes(genomes, config):
    
    print(genomes)
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes.items():
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xor_inputs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(xor_outputs)))

config = base_config
saveLocation = './'
maxNGenerations = 10
p = instantiate_population(config, xor_inputs, xor_outputs, saveLocation)
# Run for up to nGenerations generations.
winner = p.run(eval_genomes, maxNGenerations, nEpochs = 10)

g = p.best_genome


The function - Backprop everything - has just started at 1644191135.075989
The function - Backprop everything - took 0.0022101402282714844 seconds to complete
The function - generationStart - has just started at 1644191135.078888

 ****** Running generation 0 ****** 

The function - generationStart - took 5.1021575927734375e-05 seconds to complete
The function - pre_backprop - has just started at 1644191135.0789561
The function - pre_backprop - took 0.000125885009765625 seconds to complete
The function - backprop - has just started at 1644191135.079204
about to start backprop with 10 epochs
mean improvement: -0.0036812340442136616
best improvement: -0.005863359903042431
best loss: 0.49854945275039186
The function - backprop - took 0.21310997009277344 seconds to complete
The function - post_backprop - has just started at 1644191135.292331
The function - post_backprop - took 2.193450927734375e-05 seconds to complete
The function - evaluate fitness - has just started at 1644191135.292383


In [31]:
print(g)

Key: 444
Fitness: 1.444015622138977
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.06871293762612656, response=1.0, activation=sigmoid, aggregation=sum)
	9 DefaultNodeGene(key=9, bias=-0.48988294255555775, response=1.0, activation=sigmoid, aggregation=sum)
	29 DefaultNodeGene(key=29, bias=0.2887853155961029, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=0.6667834746378754, enabled=False)
	DefaultConnectionGene(key=(-2, 29), weight=0.9906593368795087, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=0.3979807758525679, enabled=False)
	DefaultConnectionGene(key=(-1, 9), weight=0.501132279010875, enabled=True)
	DefaultConnectionGene(key=(9, 0), weight=0.3039662000170686, enabled=True)
	DefaultConnectionGene(key=(29, 0), weight=0.03024116729371016, enabled=True)


In [32]:
net = nneat(g, p.config, criterion=nn.BCEWithLogitsLoss())


In [33]:
net.genome.connections
# Error is in inputs not being updated with extra info from skip layers

{(-1, 0): <neat.genes.DefaultConnectionGene at 0x7fca587c3bb0>,
 (-2, 0): <neat.genes.DefaultConnectionGene at 0x7fca587c3c10>,
 (-1, 9): <neat.genes.DefaultConnectionGene at 0x7fca587c3c70>,
 (9, 0): <neat.genes.DefaultConnectionGene at 0x7fca587c3cd0>,
 (-2, 29): <neat.genes.DefaultConnectionGene at 0x7fca587c3e50>,
 (29, 0): <neat.genes.DefaultConnectionGene at 0x7fca587c3eb0>}

In [34]:
net.forward(torch.tensor( xor_inputs[:10], dtype=torch.float64) ) [:10]

tensor([[0.5229],
        [0.5125],
        [0.5122],
        [0.5154],
        [0.5233],
        [0.5192],
        [0.5194],
        [0.5112],
        [0.5240],
        [0.5126]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [35]:
net.forward(torch.tensor( xor_inputs, dtype=torch.float64) )[:20]

tensor([[0.5229],
        [0.5125],
        [0.5122],
        [0.5154],
        [0.5233],
        [0.5192],
        [0.5194],
        [0.5112],
        [0.5240],
        [0.5126],
        [0.5245],
        [0.5132],
        [0.5241],
        [0.5132],
        [0.5134],
        [0.5125],
        [0.5101],
        [0.5138],
        [0.5105],
        [0.5139]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [36]:
# net.optimiser.zero_grad()

In [37]:
# net.optimise(xor_inputs, xor_outputs)

In [38]:
optimiser = optim.Adadelta(net.parameters())

In [39]:
net.weights

{2: Parameter containing:
 tensor([[0.0000],
         [0.0000],
         [0.3040],
         [0.0302]], dtype=torch.float64, requires_grad=True),
 1: Parameter containing:
 tensor([[0.5011, 0.0000],
         [0.0000, 0.9907]], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)}

In [40]:
optimizer = optimiser
losses = []
for i in range(10000):
    preds = net.forward(torch.tensor( xor_inputs, dtype=torch.float64) )
    loss = F.mse_loss(preds, torch.tensor( xor_outputs, dtype=torch.float64)).sqrt()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    losses.append(loss)  
losses[-10:]


[tensor(0.3475, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3475, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3475, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3475, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3475, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3475, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3475, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3475, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3475, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3475, dtype=torch.float64, grad_fn=<SqrtBackward0>)]

In [41]:
preds[:10]

tensor([[0.2095],
        [0.6267],
        [0.9924],
        [0.4982],
        [0.7643],
        [0.1312],
        [0.5188],
        [0.1311],
        [0.1330],
        [0.9939]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [42]:
net.weights

{2: Parameter containing:
 tensor([[-5.3503],
         [ 5.7057],
         [15.8553],
         [-0.1828]], dtype=torch.float64, requires_grad=True),
 1: Parameter containing:
 tensor([[ 3.1774, -0.2728],
         [-3.1908,  0.6125]], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)}

In [43]:
net.layers

{2: {'nodes': {0: {'depth': 2,
    'output_ids': [],
    'input_ids': [-1, -2, 9, 29],
    'depths': [],
    'output_layers': [],
    'needs_skip': False,
    'id': 0,
    'input_layers': [0, 0, 1, 1],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': True,
  'is_input_layer': False,
  'layer_type': 'OUTPUT',
  'input_layers': [0, 1],
  'input_shape': 4,
  'weights_shape': (4, 1),
  'out_weights': [],
  'bias': [-0.06871293762612656],
  'in_weights': [[0, 0]],
  'input_weights': array([[0.        ],
         [0.        ],
         [0.3039662 ],
         [0.03024117]]),
  'input_map': {(9, 0): (2, 0), (29, 0): (3, 0)}},
 1: {'nodes': {9: {'depth': 1,
    'output_ids': [0],
    'input_ids': [-1],
    'depths': [],
    'output_layers': [2],
    'needs_skip': False,
    'id': 9,
    'input_layers': [0],
    'skip_layer_input': False,
    'layer_index': 0},
   29: {'depth': 1,
    'output_ids': [0],
    'input_ids': [-2],
    'depths': [],
    'output_layers': [2],


In [44]:
net.node_tracker[0]['depth']

2

In [45]:


reached_nodes = []

node_stack = []

# Instantiate stack with depth==0 nodes
for node_id, node in net.node_tracker.items():
    if node_id < 0:
        node_stack.append(node_id)

while len(node_stack) > 0:
    reached_nodes.append(node_stack[0])
    for node_id in net.node_tracker[node_stack[0]]['output_ids']:
        node_stack.append(node_id)
    del(node_stack[0])

if True:
    print("the reached nodes are")
    print(reached_nodes)


for node_id in net.node_tracker:
    if not node_id in reached_nodes:
        print(reached_nodes)
        print(net.node_tracker)
        print("I can't reach this node {}".format(node_id))

the reached nodes are
[-1, -2, 0, 9, 0, 29, 0, 0]


In [46]:
print(net.genome)

Key: 444
Fitness: 1.444015622138977
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.06871293762612656, response=1.0, activation=sigmoid, aggregation=sum)
	9 DefaultNodeGene(key=9, bias=-0.48988294255555775, response=1.0, activation=sigmoid, aggregation=sum)
	29 DefaultNodeGene(key=29, bias=0.2887853155961029, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=0.6667834746378754, enabled=False)
	DefaultConnectionGene(key=(-2, 29), weight=0.9906593368795087, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=0.3979807758525679, enabled=False)
	DefaultConnectionGene(key=(-1, 9), weight=0.501132279010875, enabled=True)
	DefaultConnectionGene(key=(9, 0), weight=0.3039662000170686, enabled=True)
	DefaultConnectionGene(key=(29, 0), weight=0.03024116729371016, enabled=True)


In [47]:
print(net.genome.connections[(-1,0)])

DefaultConnectionGene(key=(-1, 0), weight=0.3979807758525679, enabled=False)


In [48]:
net.node_mappings.node_mappings

{0: {'depth': 2,
  'output_ids': [],
  'input_ids': [9, 29],
  'depths': [2, 2],
  'on_path_to_output': True,
  'on_path_to_input': False,
  'is_input': False,
  'is_output': True,
  'is_valid': True},
 9: {'depth': 1,
  'output_ids': [0],
  'input_ids': [-1],
  'depths': [1],
  'on_path_to_output': True,
  'on_path_to_input': True,
  'is_input': False,
  'is_output': False,
  'is_valid': True},
 29: {'depth': 1,
  'output_ids': [0],
  'input_ids': [-2],
  'depths': [1],
  'on_path_to_output': True,
  'on_path_to_input': True,
  'is_input': False,
  'is_output': False,
  'is_valid': True},
 -1: {'depth': 0,
  'output_ids': [9],
  'input_ids': [],
  'depths': [],
  'on_path_to_output': False,
  'on_path_to_input': True,
  'is_input': True,
  'is_output': False,
  'is_valid': True},
 -2: {'depth': 0,
  'output_ids': [29],
  'input_ids': [],
  'depths': [],
  'on_path_to_output': False,
  'on_path_to_input': True,
  'is_input': True,
  'is_output': False,
  'is_valid': True}}